In [ ]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-02-03 09:21:07.065605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-03 09:21:07.065678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-03 09:21:07.247657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 09:21:07.635723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-03 09:21:10.265793: W tensorflow/compiler/tf2

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-02-03 09:21:12,559 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-02-03 09:21:12,560 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-02-03 09:21:12,564 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:140061419531392]
2025-02-03 09:21:12,564 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2025-02-03 09:21:12,565 - INFO - RenewingAuthorizer will start by using access_token with hash "47e0decef5164280a1bd647e8b0d2165ac3b53647b486d2757d982b00b9e2b09"
2025-02-03 09:21:12,566 - INFO - Executing token refresh without client credentials
2025-02-03 09:21:12,567 - INFO - Fetching new token from Globus Auth
2025-02-03 09:21:13,099 - INFO - request done (success)
2025-02-03 09:21:13,100 - INFO - RenewingAuthorizer.

You are already logged in.


In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-02-03 09:21:13,110 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-02-03 09:21:13,111 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


# Configuration

In [20]:
# RID of source dataset, if any.
source_dataset = '2-7K8W'

EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="LAC data template",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_lac.ipynb",
    workflow_type="Test Workflow"
)

# Configuration instance.
config = ExecutionConfiguration(
    datasets=[{'rid':source_dataset, 'materialize':True}],
    # Materialize set to False if you only need the metadata from the bag, and not the assets.
    assets=['2-4JR6'],
    workflow=test_workflow,
    description="Template instance of a feature creation workflow")

# Initialize execution
execution = EA.create_execution(config)


2025-02-03 10:02:29,434 - INFO - Configuration validation successful!
2025-02-03 10:02:30,650 - INFO - Initializing downloader: GenericDownloader v1.7.5 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-02-03 10:02:30,652 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-02-03 10:02:30,665 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-02-03 10:02:30,666 - INFO - Validating credentials for host: dev.eye-ai.org
2025-02-03 10:02:30,678 - INFO - Creating bag directory: /tmp/tmpld6nw8k3/Dataset_2-7K8W
2025-02-03 10:02:30,679 - INFO - Creating bag for directory /tmp/tmpld6nw8k3/Dataset_2-7K8W
2025-02-03 10:02:30,680 - INFO - Creating data directory
2025-02-03 10:02:30,680 - INFO - Moving /tmp/tmpld6nw8k3/Dataset_2-7K8W/tmpk9ney3_4 to data
2025-02-03 10:02

ValidationError: 1 validation error for Execution.update_status
3
  Unexpected positional argument [type=unexpected_positional_argument, input_value='5-SY1P', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/unexpected_positional_argument

In [7]:
print(execution)

caching_dir: /data
working_dir: /data/lizhiwei/EyeAI_working
execution_rid: 5-SY1E
workflow_rid: 5-SGC2
dataset_paths: [PosixPath('/data/2-7K8W_4d48009a8474a2be36e1719c89fb9380115e04c65c9b3ad9939ca65c6097be40/Dataset_2-7K8W')]
asset_paths: [PosixPath('/data/lizhiwei/EyeAI_working/5-SY1E/asset/optic_disk_crop_model.hdf5')]
configuration: datasets=[DatasetSpec(rid='2-7K8W', materialize=False)] assets=['2-4JR6'] workflow=Workflow(name='LAC data template', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_lac.ipynb', workflow_type='Test Workflow', version=None, description=None) description='Template instance of a feature creation workflow'


# Create DatasetBag

In [8]:
ds_bag = DatasetBag(execution.dataset_paths[0])

In [9]:
ds_bag.list_tables()

['deriva-ml:Dataset',
 'deriva-ml:Dataset_Dataset',
 'deriva-ml:Dataset_Dataset_Type',
 'deriva-ml:Dataset_Execution',
 'deriva-ml:Dataset_Type',
 'deriva-ml:Execution',
 'deriva-ml:Execution_Asset',
 'deriva-ml:Execution_Asset_Execution',
 'deriva-ml:Execution_Asset_Type',
 'deriva-ml:Execution_Metadata',
 'deriva-ml:Execution_Metadata_Execution',
 'deriva-ml:Execution_Metadata_Type',
 'deriva-ml:Feature_Name',
 'deriva-ml:Workflow',
 'deriva-ml:Workflow_Type',
 'eye-ai:Annotation',
 'eye-ai:Annotation_Function',
 'eye-ai:Annotation_Type',
 'eye-ai:Clinical_Records',
 'eye-ai:Clinical_Records_ICD10_Eye',
 'eye-ai:Clinical_Records_Observation',
 'eye-ai:Condition_Label',
 'eye-ai:Diagnosis_Image',
 'eye-ai:Diagnosis_Observation',
 'eye-ai:Diagnosis_Status',
 'eye-ai:Diagnosis_Subject',
 'eye-ai:Diagnosis_Tag',
 'eye-ai:Execution_Image_Fundus_Angle',
 'eye-ai:Execution_Image_Fundus_Laterality',
 'eye-ai:Fundus_Bounding_Box',
 'eye-ai:ICD10_Eye',
 'eye-ai:Image',
 'eye-ai:Image_Angle',
 

In [18]:
ds_bag.get_table_as_dataframe('Subject')

eye-ai:Subject


,RID,RCT,RMT,RCB,RMB,Subject_ID,Subject_Gender,Subject_Ethnicity
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-10-29 05:02:32.50148+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_1,F,Caucasian
1,2-7KTT,2024-04-15 23:44:22.184165+00,2024-10-29 05:02:32.50148+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_2,M,Caucasian
2,2-7KTW,2024-04-15 23:44:22.184165+00,2024-10-29 05:02:32.50148+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_3,M,African Descent
3,2-7KTY,2024-04-15 23:44:22.184165+00,2024-10-29 05:02:32.50148+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_4,F,Latin American
4,2-7KV0,2024-04-15 23:44:22.184165+00,2024-10-29 05:02:32.50148+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_5,M,African Descent
5,6RDP,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other
6,6RE0,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_900811,M,Asian
7,6REA,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_963487,M,African Descent
8,6RF0,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_428549,M,Latin American
9,6RGY,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_612937,F,Latin American


In [16]:
# get dataframe of all the values in Image
ds_bag.get_table_as_dataframe('Image')

eye-ai:Image


,RID,RCT,RMT,RCB,RMB,URL,Filename,Description,Length,MD5,Observation,Image_ID,Source_Image,Process,Image_Side,Image_Angle,Image_Tag
0,2-BDD4,2024-05-30 03:25:26.399943+00,2024-11-22 22:00:41.329655+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/multimodal/scans/subject/2/obse...,/data/2-AGAW_66da02d10d7250c75a172519c233d7459...,,6273823,7c9636823623cf55f8df86cb90c3c5c9,2-7RGW,Multimodal_39,,,Unknown,Unknown,
1,2-BDNG,2024-05-30 03:29:34.647629+00,2024-11-22 22:00:41.329655+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/multimodal/scans/subject/4/obse...,/data/2-AGAW_66da02d10d7250c75a172519c233d7459...,,561632,cf5f2d6c7b41ae267dda6d3cbabed8f2,2-7RH4,Multimodal_171,,,Unknown,Unknown,
2,2-BDY4,2024-05-30 03:33:58.535376+00,2024-11-22 22:00:41.329655+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/multimodal/scans/subject/2/obse...,/data/2-AGAW_66da02d10d7250c75a172519c233d7459...,,6109112,ae516abfbd524094248543a374002f57,2-7RGW,Multimodal_306,,,Unknown,Unknown,
3,2-BE0E,2024-05-30 03:35:06.676527+00,2024-11-22 22:00:41.329655+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/multimodal/scans/subject/4/obse...,/data/2-AGAW_66da02d10d7250c75a172519c233d7459...,,465640,b38974e95c6692d7b8863e90e5fe239c,2-7RH4,Multimodal_343,,,Unknown,Unknown,
4,2-BETR,2024-05-30 03:48:30.328426+00,2024-11-22 22:00:41.329655+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/multimodal/scans/subject/3/obse...,/data/2-AGAW_66da02d10d7250c75a172519c233d7459...,,5427054,4206904e5ae8c2908db59be4e2780b25,2-7RH2,Multimodal_761,,,Unknown,Unknown,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,AFE4,2023-05-03 11:06:08.503954+00,2024-10-18 18:06:29.51325+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/966607/observatio...,/data/2-7K8W_4d48009a8474a2be36e1719c89fb93801...,,1092407,c4fce460805c4d5259e58a85992707fc,76W0,14009166,,,Right,3,Source
104,AFE6,2023-05-03 11:06:09.824986+00,2024-10-18 18:06:29.51325+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/966607/observatio...,/data/2-7K8W_4d48009a8474a2be36e1719c89fb93801...,,1132486,3c8fa7e32cc3131b14a61a5df1e318ff,76W0,14009169,,,Right,1,Source
105,AFE8,2023-05-03 11:06:11.120933+00,2024-10-18 18:06:29.51325+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/966607/observatio...,/data/2-7K8W_4d48009a8474a2be36e1719c89fb93801...,,1139738,5b5cef4d6b4b4b498c35354944ef32b2,76W0,14009170,,,Left,2,Source
106,AFEA,2023-05-03 11:06:12.387196+00,2024-10-18 18:06:29.51325+00,https://auth.globus.org/b2541312-d274-11e5-913...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/scans/subject/966607/observatio...,/data/2-7K8W_4d48009a8474a2be36e1719c89fb93801...,,1142173,2372dfe3177ac91ef300209fc535fdfe,76W0,14009176,,,Right,2,Source


In [23]:
ds_bag.get_table_as_dataframe('Observation')

eye-ai:Observation


,RID,RCT,RMT,RCB,RMB,Observation_ID,Subject,hba1c,glaucoma_hx,visual_acuity_right,...,additional_comments,return_time_frame,referral_status_time_frame,Subject_hypertension,Subject_insulin_dependent,Subject_pregnant,Subject_cataract,Subject_maculopathy,Subject_other,Subject_image_quality
0,2-7RGP,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_1,2-7KTR,,,,...,,,,,,,,,,
1,2-7RGR,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_2,2-7KTR,,,,...,,,,,,,,,,
2,2-7RGT,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_659,2-7KTT,,,,...,,,,,,,,,,
3,2-7RGW,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_660,2-7KTT,,,,...,,,,,,,,,,
4,2-7RGY,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_658,2-7KTT,,,,...,,,,,,,,,,
5,2-7RH0,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_897,2-7KTW,,,,...,,,,,,,,,,
6,2-7RH2,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_896,2-7KTW,,,,...,,,,,,,,,,
7,2-7RH4,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_1137,2-7KTY,,,,...,,,,,,,,,,
8,2-7RH6,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_1136,2-7KTY,,,,...,,,,,,,,,,
9,2-7RH8,2024-04-15 23:50:33.32379+00,2024-11-15 22:01:46.085146+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_1368,2-7KV0,,,,...,,,,,,,,,,


In [ ]:
asset = pd.read_csv(execution.asset_paths[0])

# Methods for LAC data

### Filter on angle 2 image

In [ ]:
angle2_image = EA.filter_angle_2(ds_bag)

### Diagnosis manipulation

In [10]:
init_diag = EA.image_tall(ds_bag, 'Initial Diagnosis')
grader_diag = EA.image_tall(ds_bag, 'AI_glaucomasuspect_test')
long, wide = EA.reshape_table([init_diag, grader_diag], 'Diagnosis_Image')

eye-ai:Subject
eye-ai:Observation
eye-ai:Image
eye-ai:Image_Diagnosis
eye-ai:Subject
eye-ai:Observation
eye-ai:Image
eye-ai:Image_Diagnosis


In [19]:
init_diag

,Subject_RID,Image_RID,Diagnosis_RID,Full_Name,Image_Side,Diagnosis_Image,Cup_Disk_Ratio,Image_Quality
78,6RDP,9ZYM,5-NKN0,Initial Diagnosis,Left,Suspected Glaucoma,,Good
96,6RDP,9ZYR,5-NKN4,Initial Diagnosis,Right,Suspected Glaucoma,,Good
116,6RE0,AAZ6,5-NYNJ,Initial Diagnosis,Left,Suspected Glaucoma,,Good
135,6RE0,AAZC,5-NYNR,Initial Diagnosis,Right,Suspected Glaucoma,,Good
157,6REA,AF76,5-P2X4,Initial Diagnosis,Right,Suspected Glaucoma,,Good
175,6REA,AF7A,5-P2X8,Initial Diagnosis,Left,Suspected Glaucoma,,Good
195,6RF0,9F04,5-N2PJ,Initial Diagnosis,Right,Suspected Glaucoma,,Good
212,6RF0,9F06,5-N2PM,Initial Diagnosis,Left,Suspected Glaucoma,,Good
230,6RGY,8DWG,5-M1VR,Initial Diagnosis,Left,No Glaucoma,,Good
247,6RGY,8DWJ,5-M1VT,Initial Diagnosis,Right,No Glaucoma,,Good


### Crop Image by bounding box


In [ ]:
execution.dataset_paths[0]

In [ ]:
output_dir = execution.working_dir
image_path, cropped_csv = EA.create_cropped_images(execution.dataset_paths[0], ds_bag, output_dir, crop_to_eye=True)

# Execution of ML

In [ ]:
with execution.execute() as exec:
    print("An ML Execution.") 

# Upload results

In [ ]:
# crete asset path
asset_type_name = ""
asset_path = execution.execution_asset_path(asset_type_name)
# save assets to asset_path
df.to_csv(asset_path+'filename.csv', index=False)

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)